In [0]:
--execute immediate 'alter session enable parallel dml';
--execute immediate 'truncate table stg_sma_case_phase_properties_theCase';

--select * from gap_catalog.ads_etl_owner.stg_sma_case_phase_properties_theCase;

INSERT INTO GAP_CATALOG.ADS_ETL_OWNER.STG_SMA_CASE_PHASE_PROPERTIES_THECASE
SELECT 
    cidla, 
    get_json_object(JSONDATA, '$.theCase.casePhase') AS casephase, 
    timestamp  
FROM (select cidla, type, timestamp, JSONDATA from gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full s where s.SYS_EFFECTIVE_DATE = '2025-09-01' AND get_json_object(JSONDATA, '$.theCase.casePhase') IS NOT NULL 
 AND s.type IN (
    'CASE_CREATED',
    'CASE_COMPLETED',
    'CASE_WAITING',
    'CASE_RUNNING',
    'CASE_UPDATED',
    'CASE_PHASE_CHANGED',
    'CASE_RECONCILE',
    'CASE_TRANSITIONS_EVALUATED'
) 
 ) s
LEFT JOIN (select case_key,case_source_id,cstat_key from  gap_catalog.ads_owner.cases c
WHERE c.case_valid_to = DATE'3000-01-01'
    AND c.case_source_system_id = 'SMA' 
    AND c.case_source_sys_origin = 'SMA_MONITOR_EVENTS') c
    ON c.case_source_id = s.cidla 
    AND s.type = 'CASE_TRANSITIONS_EVALUATED'
LEFT JOIN gap_catalog.ads_owner.case_status cs 
    ON c.cstat_key = cs.cstat_key 
    AND cs.cstat_valid_to = DATE'3000-01-01'
WHERE (c.case_key IS NULL OR cs.CSTAT_SOURCE_ID = get_json_object(jsondata, '$.theCase.caseStatus')) 
/*
SELECT cidla, json_value(JSONDATA, '$.theCase.casePhase' returning varchar2(120)) as casephase, timestamp  
FROM gap_catalog.ads_etl_owner.DLK_ADS_sma_monitor_events_full s
left join ads_owner.cases c on c.case_source_id=s.cidla and c.case_valid_to=date'3000-01-01' and c.case_source_system_id='SMA' and c.case_source_sys_origin='SMA_MONITOR_EVENTS' and s.type='CASE_TRANSITIONS_EVALUATED'
left join ads_owner.case_status cs on c.cstat_key=cs.cstat_key and cs.cstat_valid_to=date'3000-01-01'
where s.type in ('CASE_CREATED',
    'CASE_COMPLETED',
    'CASE_WAITING',
    'CASE_RUNNING',
    'CASE_UPDATED',
    'CASE_PHASE_CHANGED',
    'CASE_RECONCILE',
    'CASE_TRANSITIONS_EVALUATED') 
  and json_value(JSONDATA, '$.theCase.casePhase') is not null 
  and (c.case_key is null or cs.CSTAT_SOURCE_ID=json_value(jsondata, '$.theCase.caseStatus')) 
  and s.SYS_EFFECTIVE_DATE = '2025-09-01'
;
*/


